# Rogue NPC stats

**Notes:**  
The purpose of the file is to create export table of the characteristics of the game space combat objects

In [346]:
import pandas as pd
import numpy as np
import json
import os
import re
from bs4 import BeautifulSoup
import jsonpath_ng as jp
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 300)
import warnings
import openpyxl
warnings.filterwarnings('ignore')
import flatten_json as fj

### Functions

In [347]:
# Fuction to show missed data in a table
def missing_values_tab(df):
    #Count missed values
    mis_val = df.isnull().sum()
    #Count share of missed values of all data 
    mis_val_percent = round(100 * df.isnull().sum() / len(df),2)
    
    mv_table = pd.concat([mis_val, mis_val_percent], axis=1)
    
    mv_table = mv_table.rename(columns = {0 : 'missed_values', 1 : '%_of_all'})
    
    mv_table['data_type'] = df.dtypes
    mv_table = mv_table[mv_table.iloc[:,1]!=0].sort_values(by='missed_values',ascending=False)
    print ("Dataframe contains " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " strings.\n")
    print("It has  " + str(mv_table.shape[0]) +" columns with missed values")    
    return display(mv_table) 

In [348]:
def column_name_formatter(col):
    """used for delete prefixes 'Data_m_' in columns' names
    """ 
    if 'Data' in col:
        parts = col.split('_')
        new_name = parts[-1]
        return new_name
    else:
        return col

In [349]:
def show_uniques(df):
    for col in df.columns[1:-1]:
        try:
            print(col, ':    ', df[col].unique(),'\n')
        except:
            continue

In [350]:
# Функция для очистки строк от префикса '!bp_'
def clear_rows(row):
        
    if re.match('^\!bp_\w+', str(row)):
        tmp = row.split('_')
        new_value = tmp[1]        
        return new_value
    else:
        return row

In [351]:
#Function to extract json into dataframes
def create_df(filelist, datatype = None):
    recipes_df = pd.DataFrame()
    filelist_selected = []
    dfs_list = []
    for file in filelist:
        with open(file) as user_file:
            file_contents = user_file.read()
            data = json.loads(file_contents)
            dic = (fj.flatten(data))
            if datatype == None:
                df = pd.DataFrame.from_dict(dic, orient='index')
                dfs_list.append(df)
            else:
                if data['Data']['$type']==datatype:
                    df = pd.DataFrame.from_dict(dic, orient='index')
                    dfs_list.append(df)
                    filelist_selected.append(file)
    for df in dfs_list:
        recipes_df = recipes_df.join(df, how = 'outer',lsuffix = 'x_',rsuffix = 'y_')
    #assign new column names
    column_numbers = list(range(len(recipes_df.columns)))
    recipes_df.columns = column_numbers
    #fill Nans
    #recipes_df = recipes_df.fillna(0)
    #remove prefix '!bp_'
    for col in recipes_df.columns:
        recipes_df[col] = recipes_df[col].apply(clear_rows)  
    #transpose dataframe
    recipes_df = recipes_df.T
    #-------------------------------------
    cols_to_drop = []
    for col in recipes_df.columns:
        if 'Visual' in col or 'FXSettings' in col: #drop columns with visual parameters
            cols_to_drop.append(col)
        elif 'Data_Comment' in col or 'PrototypeLink' in col:  #drop columns with dev comments
            cols_to_drop.append(col)
        elif 'Meta_ShadowDeleted' in col: #drop trash column
            cols_to_drop.append(col)
        elif 'JsonPath' in col or 'PropertyPath' in col or 'OwnerString' in col or 'assetguid' in col: #drop columns with fileways
            cols_to_drop.append(col)
        elif 'UI' in col or 'Sound' in col or 'Icon' in col: #drop columns with sound, ui properties
            cols_to_drop.append(col)
    recipes_df = recipes_df.drop(cols_to_drop, axis = 1)
    if datatype is not None:
        #add column Filepath
        recipes_df['Filepath'] = filelist_selected
    else:
        recipes_df['Filepath'] = filelist
    return recipes_df

In [352]:
# Function to get rid of characters between curly brackets
""" Sometimes not working correctly
def format_description(row):
    if '{' in row:
        row = re.sub(r'\s*{.*}\s*','',row)
    return row
"""
def format_description(row):      
    openining_braces = '\{'
    closing_braces = '\}'
    text_between = '.*?'
    x = re.sub(f'[{openining_braces}]{text_between}[{closing_braces}]','',row)
    #get rid of html tags
    x = re.sub('<[^>]*>',"",x)
    return x

In [353]:
#Function to find columns with string keys
def match_pattern(row):
        if re.match('^\w{8}-\w{4}-\w{4}-\w{4}-\w{12}$', str(row)):
            return True   
        else:
            return False
   
        
match_pattern('ca24785c24384d142a6c22d7585bb3a6, BlueprintWeaponEnchantment')
"""
#finding columns with string keys
stringkey_cols = []
for col in weapon_enchants_df.columns:    
        a = weapon_enchants_df[col].apply(match_pattern).any()
        if a:            
            stringkey_cols.append(col)
print(stringkey_cols)   
    """

'\n#finding columns with string keys\nstringkey_cols = []\nfor col in weapon_enchants_df.columns:    \n        a = weapon_enchants_df[col].apply(match_pattern).any()\n        if a:            \n            stringkey_cols.append(col)\nprint(stringkey_cols)   \n    '

In [354]:
#Function to find columns with asset ids
def find_assetIds_cols(row):
        if re.match('^[a-zA-Z0-9]{32}$', str(row)):
            return True   
        else:
            return False      

"""
#finding columns with asset ids
assetIds_cols = []
for col in weapon_enchants_df.columns:    
        a = weapon_enchants_df[col].apply(find_assetIds_cols).any()
        if a:            
            assetIds_cols.append(col)
print(assetIds_cols)   
    """

'\n#finding columns with asset ids\nassetIds_cols = []\nfor col in weapon_enchants_df.columns:    \n        a = weapon_enchants_df[col].apply(find_assetIds_cols).any()\n        if a:            \n            assetIds_cols.append(col)\nprint(assetIds_cols)   \n    '

In [355]:
#Function to find columns with context actions, containing $ signs
def find_context_columns(row):
        if re.match('^\$[A-Za-z0-9]+\$\w{8}-\w{4}-\w{4}-\w{4}-\w{12}$', str(row)):
            return True   
        else:
            return False
"""
#find columns with rows containing $$
context_cols = []
for col in weapon_enchants_df.columns:    
        a = weapon_enchants_df[col].apply(find_context_columns).any()
        if a:            
            context_cols.append(col)
print(context_cols)   
    """

'\n#find columns with rows containing $$\ncontext_cols = []\nfor col in weapon_enchants_df.columns:    \n        a = weapon_enchants_df[col].apply(find_context_columns).any()\n        if a:            \n            context_cols.append(col)\nprint(context_cols)   \n    '

In [356]:
def find_item_type(row):
    """Function to return item type from the column "Data_$type"
    """
    tmp = str(row).split(',')
    data_type = tmp[1]    
    parts = re.split('(?<=[a-z])(?=[A-Z])', data_type)
    item_type = parts[-1]   
    return item_type

In [357]:
def create_text_columns(df, index_df, columns_in, column_out, format_text=False):
    """ Function to create text columns with names and descriptions got from 
        stringkeys in index file in the form of dict
        """
    # list of columns with string keys of items' names
    cols_names = columns_in
    #created column
    column_out = str(column_out )
    index_df = index_df # index_dict object got from parsing json file with text information
    index_dict = dict(zip(index_df['index'],index_df['text'])) #dict for mapping text information between two tables    
    
    #_____________________________Get items names and descriptions__________________________ 
    
    for col in cols_names:
        list_of_ids = df[col].unique() # list of stringkeys
        #filter out empty elements
        list_of_ids = list(filter(None,list_of_ids))
        for id in list_of_ids:
            try: #if stringkey is in index file return text
                df.loc[df[col]==id, column_out] = df[col].map(index_dict)
            except: #if not return empty row
                df.loc[df[col]==id, column_out] = ''          
    
         # fill missed values with empty row       
    df[column_out] = df[column_out].fillna('')
    # format description to get rid of {}
    if format_text == True:
        df[column_out] = df[column_out].apply(format_description)

   
    return df

In [358]:
def find_property_cols(row):
    #if  re.match('^Data_Components_[0-9]_\$type$',row) or re.match('^Data_Components_[0-9]_m_(?:Fact|Facts)(?:$|_[0-9]$)',row):   
    if re.match('^Data_Components_[0-9]_(?:\$type$|m_(?:Fact|Facts)(?:$|_[0-9]$))',str(row)):
        return True
    else:
        return False

"""
#find columns with rows containing property values
property_cols = []
for col in equip_items_df.columns:    
        a = equip_items_df[col].apply(find_property_cols).any()
        if a:            
            property_cols.append(col)
print(property_cols)   
    """    

'\n#find columns with rows containing property values\nproperty_cols = []\nfor col in equip_items_df.columns:    \n        a = equip_items_df[col].apply(find_property_cols).any()\n        if a:            \n            property_cols.append(col)\nprint(property_cols)   \n    '

In [359]:
def find_properties(row):
    #if  re.match('^Data_Components_[0-9]_\$type$',row) or re.match('^Data_Components_[0-9]_m_(?:Fact|Facts)(?:$|_[0-9]$)',row):   
    if re.match('^Data_Components_[0-9]_(?:\$type$|m_[A-Za-z]+(?:$|_[0-9]$))',str(row)):
        return True
    else:
        return False
def select_columns(df, keywords):
    """function to select columns to keep from a dataframe and return 4 lists of
        stringkey columns, assetids columns, misc columns and combined list of columns
    Arguments: df - dataframe to choose columns from, 
                keywords - list of keywords or names of columns to select for misc list    
    """
    selected_cols = [] # list of needed columns
    assetIds_cols = [] #list of columns with asset ids
    stringkey_cols = [] # list of columns with string keyes
    misc_cols = [] # list of misc columns
    property_cols = [] # list of property columns
    intro_cols = []   # first columns: AssetId and Data_$type in any table

    def find_ids(col):
        if re.match('^[a-z0-9]{32}(?:$|[, A-Za-z])', str(col)):
            return True
        else:
            return False      

    #find columns with asset ids
    for col in df.columns:
        if col !='AssetId':
            a = df[col].apply(find_assetIds_cols).any()
            if a:            
                assetIds_cols.append(col)
        else:
            intro_cols.append(col)

    #find columns with string keys
    for col in df.columns:    
        a = df[col].apply(match_pattern).any()
        if a:            
            stringkey_cols.append(col)
    
    #find columns with rows containing property values

    for col in df.columns: 
        # if name of the column is according to the pattern   
        if  re.match('^Data_Components_[0-9]_(?:\$type$|m_[A-Za-z]+(?:$|_[0-9]$))',str(col)):
            a = df[col].apply(find_ids).any()
            if a:                        
        #and its values are assetIds keys  add to property list                 
                property_cols.append(col) 
      
    # find miscellanious columns
    if len(keywords)>0:
        keywords = keywords
        for keyword in keywords:
            for col in df.columns:
                if keyword in col:
                    misc_cols.append(col)
    #else:
        #misc_cols = [d for d in df.columns if d not in assetIds_cols if d not in stringkey_cols if d not in property_cols]
    #update assetIds_cols
    assetIds_cols = [d for d in assetIds_cols if d not in property_cols]
    #Must- have columns, 'AssetId' and 'Data_$type'
    intro_cols.append(df.columns[1]) #add Data_$type
    #create combined list            
    selected_cols = intro_cols + assetIds_cols+ property_cols + stringkey_cols + misc_cols + ['Filepath']
      
    return selected_cols, assetIds_cols, stringkey_cols, property_cols, misc_cols

In [360]:
def find_property_cols(row):
    #if  re.match('^Data_Components_[0-9]_\$type$',row) or re.match('^Data_Components_[0-9]_m_(?:Fact|Facts)(?:$|_[0-9]$)',row):   
    if re.match('^Data_Components_[0-9]_(?:\$type$|m_(?:Fact|Facts)(?:$|_[0-9]$))',str(row)):
        return True
    else:
        return False

"""
#find columns with rows containing property values
property_cols = []
for col in equip_items_df.columns:    
        a = equip_items_df[col].apply(find_property_cols).any()
        if a:            
            property_cols.append(col)
print(property_cols)   
    """    
       

'\n#find columns with rows containing property values\nproperty_cols = []\nfor col in equip_items_df.columns:    \n        a = equip_items_df[col].apply(find_property_cols).any()\n        if a:            \n            property_cols.append(col)\nprint(property_cols)   \n    '

In [361]:
def find_item_columns(df):
    #Function to find columns with names like Data_Items_0_m_Item
    item_cols = []
    for col in df.columns:
        try:
            if re.match('^Data_Items_[0-9]+_m_Item$',str(col)):
                item_cols.append(col)
        except Exception as err:
                    print(f'Unexpected {err=}, {type(err)=}',col)
            
    return item_cols

In [362]:
def drop_trash_columns (df):
    df = df.loc[:,~df.where(df.astype(bool)).isna().all(axis = 0)]
    return df

In [363]:
def get_text_info(df, map_df, index_df, columns_in, keylist = None, return_temp = False, suffix = None, format_text=False, add_aggr_col = False):
    """ Function to create text columns with names and descriptions. Works on columns with asset ids
        and get filenames of those asset ids in asset_df and then look into files to extract strinkeys,
        max length of keylist is 4.
        Arguments:
        df - target df apply changes to;
        map_df - assets_df to get assets ids of files;
        index_df - dataframe of index file to get text info from;
        columns_in - list of columns apply changes over;
        keylist - keys of the received file get information from, if None search string keys to get ONLY names (not description keys);
        return_temp - if True return list of first columns with data, skipping further search and creation of columns;
                     if False, intermediate columns will be dropped.
        suffix - assign siffix to names of intermediate columns, if you return them, its better dont use 'text' for it,
                 because it is used inside function.
        format_text - if True apply format_description() function to the columns;
        add_aggr_col - create column with sum of values (or concatenation of strings) for provided columns;       
        """    
    # list of columns with string keys of items' names
    cols_names = columns_in
    #created column        
    # map_df - a dataframe for mapping    
    assets_dict = dict(zip(map_df.iloc[:,0],map_df.iloc[:,1])) #dict for mapping text information between two tables
    #reference to dataframe go from localization file    
    
    def find_parent(D):
        """find first key path to value containing stringkey"""    
        for k, v in D.items():        
            if isinstance(v,dict):
                parent = find_parent(v)
                if parent:
                    return [k] + parent
            elif match_pattern(v):                
                return [k]
    def find_stringkeys(row, keylist = None):    
        try:
            row = row.strip()
            with open(row) as asset_file:           
                file_contents = asset_file.read()
                data = json.loads(file_contents)
                try: 
                    if keylist is None:                   
                        keylist = find_parent(data)
                        if len(keylist)==3:
                            key = data[keylist[0]][keylist[1]][keylist[2]]
                            return key
                        else:
                            key = data[keylist[0]][keylist[1]][keylist[2]][keylist[3]]
                            return key
                    else:
                        if len(keylist)==1:
                            key = data[keylist[0]]
                            return key
                        if len(keylist)==2:
                            key = data[keylist[0]][keylist[1]]
                            return key
                        if len(keylist)==3:
                            key = data[keylist[0]][keylist[1]][keylist[2]]
                            return key
                        else:
                            key = data[keylist[0]][keylist[1]][keylist[2]][keylist[3]]
                            return key                  
                except:                         
                        return ''
                   
                                   
                
        except:
            return ''      
    
    #_____________________________Get items names and descriptions__________________________ 
    temp_list = []
    columns_out = []
    if suffix == None:
        suffix=='stringkey'
    for col in cols_names:                
                df[col] = df[col].str.strip()         
                filenames_series = df[col].map(assets_dict)               
                     
                df['{}_{}'.format(col,suffix)]  = filenames_series.apply(find_stringkeys, keylist = keylist)
                

                temp_list.append('{}_{}'.format(col,suffix))
                text_column_name = '{}_text'.format(col)
                columns_out.append(text_column_name)
    if return_temp == True:
          if format_text == True:            
            df[temp_list] = df[temp_list].apply(format_description,format_text = format_text)
            return df
          else:
              return df
    else:
         
        for i in range(len(columns_out)):
            col_in = [temp_list[i]]
            df = create_text_columns(df, index_df, col_in, columns_out[i], format_text = format_text)
        #drop created stringkey cols
        df = df.drop(columns = temp_list, axis = 1)
        if add_aggr_col == True:
            df['All_Items'] =df[columns_out].agg(sum)  
        return df, columns_out

### Folders

In [364]:
#folder with the gameobjects' parameters
work_folder = r'e:\Steam\RogueTrader\WorkFolder\Blueprints'
# file with the english localization
index_file = r'e:\Steam\RogueTrader\Localization\enGB.json'
#file with the russian localization
index_file_ru = r'e:\Steam\RogueTrader\Localization\ruRU.json'
save_folder = r'e:\Steam\RogueTrader\SaveFolder'
load_folder = r'e:\Steam\RogueTrader\LoadFolder'


## Work Part

### Load support tables

In [365]:
#loading support dataframes
assets_df = pd.read_csv(load_folder + r'\assets_df.csv')
index_df = pd.read_csv(load_folder + r'\index_df.csv')
index_df_ru = pd.read_csv(load_folder + r'\index_df_ru.csv')
#english version weapon table
weapons_table = pd.read_csv(load_folder + r'\weapons.csv')
#english version armor table
armors_df = pd.read_csv(load_folder + r'\armors_df.csv')

### Folders

In [366]:
folder_dialogs = r'E:\Steam\RogueTrader\WorkFolder\Blueprints\Units'

### Full table of units characteristics

# UNCOMMENT WHEn UPDATE
list_of_files = []
for root,dirs,files in os.walk(folder_dialogs):
        for file in files:
            list_of_files.append(os.path.join(root, file))
    

print(len(list_of_files))
units_data = create_df(list_of_files, datatype='fa4fa7e4548127a47a2846c91b051065, BlueprintUnit')
units_data = units_data.replace('[]', np.nan)
units_data = drop_trash_columns(units_data)
#Drop color columns
for col in units_data.columns:
      if 'Color' in col or 'Prefab' in col:
            units_data.drop(columns=[col], axis = 1, inplace=True)

<span style = "color:yellow">Saving units_data:</span>

In [367]:
#units_data.to_csv(save_folder + r'\npc_units_data.csv', index = False)

Loading saved ```units_data```:

In [368]:
units_data = pd.read_csv(save_folder + r'\npc_units_data.csv')

In [369]:
display(units_data.head())
display(units_data['Data_$type'].unique())

,AssetId,Data_$type,Data_Alignment,Data_ArmyType,Data_Body_DisableHands,Data_Body_m_AdditionalLimbs_0,Data_Body_m_AdditionalLimbs_1,Data_Body_m_Armor,Data_Body_m_Belt,Data_Body_m_EmptyHandWeapon,Data_Body_m_Feet,Data_Body_m_Gloves,Data_Body_m_Head,Data_Body_m_Mechadendrites_0,Data_Body_m_Mechadendrites_1,Data_Body_m_Mechadendrites_2,Data_Body_m_Mechadendrites_3,Data_Body_m_Mechadendrites_4,Data_Body_m_Mechadendrites_5,Data_Body_m_Neck,Data_Body_m_PrimaryHand,Data_Body_m_PrimaryHandAlternative1,Data_Body_m_QuickSlots_0,Data_Body_m_QuickSlots_1,Data_Body_m_QuickSlots_2,Data_Body_m_QuickSlots_3,Data_Body_m_QuickSlots_4,Data_Body_m_Ring1,Data_Body_m_Ring2,Data_Body_m_SecondaryHand,Data_Body_m_SecondaryHandAlternative1,Data_Body_m_Shoulders,Data_Body_m_Wrist,Data_Components_0_$type,Data_Components_0_AbsorptionPenetration_Property,Data_Components_0_AbsorptionPenetration_PropertyName,Data_Components_0_AbsorptionPenetration_ValueRank,Data_Components_0_AbsorptionPenetration_ValueShared,Data_Components_0_AbsorptionPenetration_ValueType,Data_Components_0_Actions_Actions_0_$type,Data_Components_0_Actions_Actions_0_Actions_Actions_0_$type,Data_Components_0_Actions_Actions_0_Actions_Actions_0_BonusDamage_Property,Data_Components_0_Actions_Actions_0_Actions_Actions_0_BonusDamage_PropertyName,Data_Components_0_Actions_Actions_0_Actions_Actions_0_BonusDamage_ValueRank,Data_Components_0_Actions_Actions_0_Actions_Actions_0_BonusDamage_ValueShared,Data_Components_0_Actions_Actions_0_Actions_Actions_0_BonusDamage_ValueType,Data_Components_0_Actions_Actions_0_Actions_Actions_0_DamageType_Type,Data_Components_0_Actions_Actions_0_Actions_Actions_0_MaxDamage_Property,Data_Components_0_Actions_Actions_0_Actions_Actions_0_MaxDamage_PropertyName,Data_Components_0_Actions_Actions_0_Actions_Actions_0_MaxDamage_ValueRank,Data_Components_0_Actions_Actions_0_Actions_Actions_0_MaxDamage_ValueShared,Data_Components_0_Actions_Actions_0_Actions_Actions_0_MaxDamage_ValueType,Data_Components_0_Actions_Actions_0_Actions_Actions_0_MinDamage_Property,Data_Components_0_Actions_Actions_0_Actions_Actions_0_MinDamage_PropertyName,Data_Components_0_Actions_Actions_0_Actions_Actions_0_MinDamage_ValueRank,Data_Components_0_Actions_Actions_0_Actions_Actions_0_MinDamage_ValueShared,Data_Components_0_Actions_Actions_0_Actions_Actions_0_MinDamage_ValueType,Data_Components_0_Actions_Actions_0_Actions_Actions_0_Penetration_BonusValue_Property,Data_Components_0_Actions_Actions_0_Actions_Actions_0_Penetration_BonusValue_PropertyName,Data_Components_0_Actions_Actions_0_Actions_Actions_0_Penetration_BonusValue_ValueRank,Data_Components_0_Actions_Actions_0_Actions_Actions_0_Penetration_BonusValue_ValueShared,Data_Components_0_Actions_Actions_0_Actions_Actions_0_Penetration_BonusValue_ValueType,Data_Components_0_Actions_Actions_0_Actions_Actions_0_Penetration_DiceCountValue_Property,Data_Components_0_Actions_Actions_0_Actions_Actions_0_Penetration_DiceCountValue_PropertyName,Data_Components_0_Actions_Actions_0_Actions_Actions_0_Penetration_DiceCountValue_ValueRank,Data_Components_0_Actions_Actions_0_Actions_Actions_0_Penetration_DiceCountValue_ValueShared,Data_Components_0_Actions_Actions_0_Actions_Actions_0_Penetration_DiceCountValue_ValueType,Data_Components_0_Actions_Actions_0_Actions_Actions_0_Penetration_DiceType,Data_Components_0_Actions_Actions_0_Actions_Actions_0_PreRolledSharedValue,Data_Components_0_Actions_Actions_0_Actions_Actions_0_ResultSharedValue,Data_Components_0_Actions_Actions_0_Actions_Actions_0_UseDiceForDamage,Data_Components_0_Actions_Actions_0_Actions_Actions_0_Value_BonusValue_Property,Data_Components_0_Actions_Actions_0_Actions_Actions_0_Value_BonusValue_PropertyName,Data_Components_0_Actions_Actions_0_Actions_Actions_0_Value_BonusValue_Value,Data_Components_0_Actions_Actions_0_Actions_Actions_0_Value_BonusValue_ValueRank,Data_Components_0_Actions_Actions_0_Actions_Actions_0_Value_BonusValue_ValueShared,Data_Components_0_Actions_Actions_0_Actions_Actions_0_Valu

array(['fa4fa7e4548127a47a2846c91b051065, BlueprintUnit'], dtype=object)

### Table of main unit characteristics

In [370]:
# list of columns with stats and its modifiers
stats = [x for x in units_data.columns if 'OldWarhammer' in x or ('Modifier' in x and 'Warhammer' in x )]
#selected_cols, assetIds_cols, stringkey_cols, property_cols, misc_cols = select_columns(units_data, stats)

In [371]:
target_columns = (['AssetId', 'Data_LocalizedName_stringkey', 'Data_DifficultyType', 'Data_m_Faction', 'Data_m_Army', 
                   'Data_m_Brain', 'Data_MaxHP'] 
                + stats + [ 'Data_Components_0_BonusAbsorptionValue_Value',
                        'Data_Components_0_BonusDeflectionValue_Value', 'Data_Components_0_DodgeChance_Value',
                        'Data_Components_1_BonusAbsorptionValue_Value', 'Data_Components_1_BonusDeflectionValue_Value',
                        'Data_Components_1_DodgeChance_Value',
                        'Data_Components_2_BonusAbsorptionValue_Value', 'Data_Components_2_BonusDeflectionValue_Value',
                        'Data_Components_2_DodgeChance_Value',
                        'Data_WarhammerInitialAPBlue', 'Data_Body_m_EmptyHandWeapon', 
                        'Data_Body_m_PrimaryHand',	 'Data_Body_m_PrimaryHandAlternative1',
                        'Data_Body_m_SecondaryHand', 'Data_Body_m_SecondaryHandAlternative1',
                        'Data_Body_m_Head', 'Data_Body_m_Armor', 'Data_Body_m_Feet',	'Data_Body_m_Gloves',
                        'Data_Body_m_Belt', 'Data_Body_m_Neck', 'Data_Body_m_Ring1',	'Data_Body_m_Ring2',	
	                    'Data_Body_m_Shoulders',	'Data_Body_m_Wrist',
                        'Data_Body_m_QuickSlots_0',	'Data_Body_m_QuickSlots_1',	'Data_Body_m_QuickSlots_2',	
                        'Data_Body_m_QuickSlots_3',	'Data_Body_m_QuickSlots_4',	
                        'Data_Body_m_Mechadendrites_0',	'Data_Body_m_Mechadendrites_1',	
                        'Data_Body_m_Mechadendrites_2','Data_Body_m_Mechadendrites_3',	
                        'Data_Body_m_Mechadendrites_4',	'Data_Body_m_Mechadendrites_5',
                        'Filepath']
                    )
units_df = units_data[target_columns]

In [372]:
units_df.head()

,AssetId,Data_LocalizedName_stringkey,Data_DifficultyType,Data_m_Faction,Data_m_Army,Data_m_Brain,Data_MaxHP,Data_OldWarhammerAgility,Data_OldWarhammerBallisticSkill,Data_OldWarhammerFellowship,Data_OldWarhammerIntelligence,Data_OldWarhammerPerception,Data_OldWarhammerStrength,Data_OldWarhammerToughness,Data_OldWarhammerWeaponSkill,Data_OldWarhammerWillpower,Data_WarhammerAgilitySetting_Modifier,Data_WarhammerBallisticSkillSetting_Modifier,Data_WarhammerFellowshipSetting_Modifier,Data_WarhammerIntelligenceSetting_Modifier,Data_WarhammerPerceptionSetting_Modifier,Data_WarhammerStrengthSetting_Modifier,Data_WarhammerToughnessSetting_Modifier,Data_WarhammerWeaponSkillSetting_Modifier,Data_WarhammerWillpowerSetting_Modifier,Data_Components_0_BonusAbsorptionValue_Value,Data_Components_0_BonusDeflectionValue_Value,Data_Components_0_DodgeChance_Value,Data_Components_1_BonusAbsorptionValue_Value,Data_Components_1_BonusDeflectionValue_Value,Data_Components_1_DodgeChance_Value,Data_Components_2_BonusAbsorptionValue_Value,Data_Components_2_BonusDeflectionValue_Value,Data_Components_2_DodgeChance_Value,Data_WarhammerInitialAPBlue,Data_Body_m_EmptyHandWeapon,Data_Body_m_PrimaryHand,Data_Body_m_PrimaryHandAlternative1,Data_Body_m_SecondaryHand,Data_Body_m_SecondaryHandAlternative1,Data_Body_m_Head,Data_Body_m_Armor,Data_Body_m_Feet,Data_Body_m_Gloves,Data_Body_m_Belt,Data_Body_m_Neck,Data_Body_m_Ring1,Data_Body_m_Ring2,Data_Body_m_Shoulders,Data_Body_m_Wrist,Data_Body_m_QuickSlots_0,Data_Body_m_QuickSlots_1,Data_Body_m_QuickSlots_2,Data_Body_m_QuickSlots_3,Data_Body_m_QuickSlots_4,Data_Body_m_Mechadendrites_0,Data_Body_m_Mechadendrites_1,Data_Body_m_Mechadendrites_2,Data_Body_m_Mechadendrites_3,Data_Body_m_Mechadendrites_4,Data_Body_m_Mechadendrites_5,Filepath
0,d0fee44fe0d4a864b819fc993dc68ba1,NaN,Swarm,d75c5993785785d468211d9a1a3c87a6,NaN,NaN,6,30,30,30,30,30,30,30,30,30,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,20375b5a0c9243d45966bd72c690ab74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,E:\Steam\RogueTrader\WorkFolder\Blueprints\Units\MapObjectKitten.jbp
1,6db1a543dd2440259670cd5213369ff7,NaN,Swarm,d64258e86eeb1d8479f35a9b16f6590a,NaN,NaN,6,30,30,30,30,30,30,30,30,30,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,20375b5a0c9243d45966bd72c690ab74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,E:\Steam\RogueTrader\WorkFolder\Blueprints\Units\MapObjectKitten_CryptArena.jbp
2,72fe2bfe437fdb24ea460b1a7c8b0041,50de44f8-4606-4055-9639-e9520f8a006b,Elite,72f240260881111468db610b6c37c099,NaN,NaN,0,30,30,30,30,30,30,30,30,30,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,d7db4bf7fc2700245a2f0dcb4f6491f3,33170608963441409d260ff82b74ae42,e3c064c362884cd699844699b478ac69,NaN,NaN,NaN,5b69b026b56c49b0b33b060b23b948bb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56e3576b84384495b9015d5ec88217ac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,E:\Steam\RogueTrader\WorkFolder\Blueprints\Units\Companions\CompanionAbelardPrototype.jbp
3,854c3386b3df29a4f8961cccb611bf39,289b1843-fe90-4cfa-84da-a391b9119ad5,Elite,72f240260881111468db610b6c37c099,NaN,NaN,0,30,30,30,30,30,30,30,30,30,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,d7db4bf7fc2700245a2f0dcb4f6491f3,2e6f4680bfcc45349d02830f518e04a3,2c8b2e940be549fbb322adff722200f0,NaN,NaN,NaN,5b69b026b56c49b0b33b060b23b948bb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56e3576b84384495b9015d5ec88217ac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,E:\Steam\RogueTrader\WorkFolder\Blueprints\Units\Companions\CompanionArgentaPrototype.jbp
4,8617540a3b554273a3557c4c898c3799,5bb02d45-bd3c-4739-b493-b99563d720ef,Elite,72f240260881111468db610b6c37c099,NaN,NaN,0,30,30,30,30,30,30,30,30,30,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,d7db4bf7fc2700245a2f0dcb4f6491f3,05159922d36a4018bdcab94a4a3f979c,c0f657dfd9ad4462906d0d4a376c52a5,NaN,NaN,NaN,d1ee6e99394d4bde9c288d5fc8bb989a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

Add npc names:

In [373]:
#get_text_info(df, map_df, index_df, columns_in, keylist = None, return_temp = False, suffix = None, format_text=False, add_aggr_col = False):
columns_in = ['Data_LocalizedName_stringkey']
units_df = create_text_columns(df = units_df,
                                index_df = index_df,
                                columns_in=columns_in,
                                column_out='Unit_Name', 
                                format_text=True)

In [374]:
units_df.head()

,AssetId,Data_LocalizedName_stringkey,Data_DifficultyType,Data_m_Faction,Data_m_Army,Data_m_Brain,Data_MaxHP,Data_OldWarhammerAgility,Data_OldWarhammerBallisticSkill,Data_OldWarhammerFellowship,Data_OldWarhammerIntelligence,Data_OldWarhammerPerception,Data_OldWarhammerStrength,Data_OldWarhammerToughness,Data_OldWarhammerWeaponSkill,Data_OldWarhammerWillpower,Data_WarhammerAgilitySetting_Modifier,Data_WarhammerBallisticSkillSetting_Modifier,Data_WarhammerFellowshipSetting_Modifier,Data_WarhammerIntelligenceSetting_Modifier,Data_WarhammerPerceptionSetting_Modifier,Data_WarhammerStrengthSetting_Modifier,Data_WarhammerToughnessSetting_Modifier,Data_WarhammerWeaponSkillSetting_Modifier,Data_WarhammerWillpowerSetting_Modifier,Data_Components_0_BonusAbsorptionValue_Value,Data_Components_0_BonusDeflectionValue_Value,Data_Components_0_DodgeChance_Value,Data_Components_1_BonusAbsorptionValue_Value,Data_Components_1_BonusDeflectionValue_Value,Data_Components_1_DodgeChance_Value,Data_Components_2_BonusAbsorptionValue_Value,Data_Components_2_BonusDeflectionValue_Value,Data_Components_2_DodgeChance_Value,Data_WarhammerInitialAPBlue,Data_Body_m_EmptyHandWeapon,Data_Body_m_PrimaryHand,Data_Body_m_PrimaryHandAlternative1,Data_Body_m_SecondaryHand,Data_Body_m_SecondaryHandAlternative1,Data_Body_m_Head,Data_Body_m_Armor,Data_Body_m_Feet,Data_Body_m_Gloves,Data_Body_m_Belt,Data_Body_m_Neck,Data_Body_m_Ring1,Data_Body_m_Ring2,Data_Body_m_Shoulders,Data_Body_m_Wrist,Data_Body_m_QuickSlots_0,Data_Body_m_QuickSlots_1,Data_Body_m_QuickSlots_2,Data_Body_m_QuickSlots_3,Data_Body_m_QuickSlots_4,Data_Body_m_Mechadendrites_0,Data_Body_m_Mechadendrites_1,Data_Body_m_Mechadendrites_2,Data_Body_m_Mechadendrites_3,Data_Body_m_Mechadendrites_4,Data_Body_m_Mechadendrites_5,Filepath,Unit_Name
0,d0fee44fe0d4a864b819fc993dc68ba1,NaN,Swarm,d75c5993785785d468211d9a1a3c87a6,NaN,NaN,6,30,30,30,30,30,30,30,30,30,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,20375b5a0c9243d45966bd72c690ab74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,E:\Steam\RogueTrader\WorkFolder\Blueprints\Units\MapObjectKitten.jbp,
1,6db1a543dd2440259670cd5213369ff7,NaN,Swarm,d64258e86eeb1d8479f35a9b16f6590a,NaN,NaN,6,30,30,30,30,30,30,30,30,30,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,20375b5a0c9243d45966bd72c690ab74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,E:\Steam\RogueTrader\WorkFolder\Blueprints\Units\MapObjectKitten_CryptArena.jbp,
2,72fe2bfe437fdb24ea460b1a7c8b0041,50de44f8-4606-4055-9639-e9520f8a006b,Elite,72f240260881111468db610b6c37c099,NaN,NaN,0,30,30,30,30,30,30,30,30,30,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,d7db4bf7fc2700245a2f0dcb4f6491f3,33170608963441409d260ff82b74ae42,e3c064c362884cd699844699b478ac69,NaN,NaN,NaN,5b69b026b56c49b0b33b060b23b948bb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56e3576b84384495b9015d5ec88217ac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,E:\Steam\RogueTrader\WorkFolder\Blueprints\Units\Companions\CompanionAbelardPrototype.jbp,Abelard
3,854c3386b3df29a4f8961cccb611bf39,289b1843-fe90-4cfa-84da-a391b9119ad5,Elite,72f240260881111468db610b6c37c099,NaN,NaN,0,30,30,30,30,30,30,30,30,30,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,d7db4bf7fc2700245a2f0dcb4f6491f3,2e6f4680bfcc45349d02830f518e04a3,2c8b2e940be549fbb322adff722200f0,NaN,NaN,NaN,5b69b026b56c49b0b33b060b23b948bb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56e3576b84384495b9015d5ec88217ac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,E:\Steam\RogueTrader\WorkFolder\Blueprints\Units\Companions\CompanionArgentaPrototype.jbp,Argenta
4,8617540a3b554273a3557c4c898c3799,5bb02d45-bd3c-4739-b493-b99563d720ef,Elite,72f240260881111468db610b6c37c099,NaN,NaN,0,30,30,30,30,30,30,30,30,30,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,d7db4bf7fc2700245a2f0dcb4f6491f3,05159922d36a4018bdcab94a4a3f979c,c0f657dfd9ad4462906d0d4a376c52a5,NaN,NaN,NaN,d1ee6e99394d4bde9c288d5fc8bb989a,NaN,

Merge ```units_df``` with ```armors_df``` to get armor name:

In [375]:
units_df = (units_df.merge(armors_df[['AssetId','Item_Name','Data_m_DamageAbsorption']], how = 'left',
                          left_on = ['Data_Body_m_Armor'],
                          right_on = ['AssetId'],
                          suffixes=(None, '_y'))
                          .drop(columns = ['AssetId_y'])
                          .rename(columns = {'Item_Name':'Armor_Name'})
                                                                    )

Merge ```units_df``` with ```weapons_table``` to get in-hand weapon names:

In [376]:
#Add primary weapon name
units_df = (units_df.merge(weapons_table[['AssetId','Item_Name']], how = 'left',
                          left_on = ['Data_Body_m_PrimaryHand'],
                          right_on = ['AssetId'],
                          suffixes=(None, '_y'))
                          .drop(columns = ['AssetId_y'])
                          .rename(columns = {'Item_Name':'PrimaryHand_Weapon'})
                                                                    )
#add secondary weapon name
units_df = (units_df.merge(weapons_table[['AssetId','Item_Name']], how = 'left',
                          left_on = ['Data_Body_m_SecondaryHand'],
                          right_on = ['AssetId'],
                          suffixes=(None, '_y'))
                          .drop(columns = ['AssetId_y'])
                          .rename(columns = {'Item_Name':'SecondaryHand_Weapon'})
                                                                    )

Creating a unique column for each 'Data_Components_{number}_{StatName}' columns: 

**Notes**:  
```Stat_Base_Value``` = ```OldWarhammerStatValue``` + ```WarhammerStatModifier```?

In [377]:
t = units_df.query('Unit_Name=="Inquisition Techsorcist"')
t[t.columns[~t.isnull().all()]]

,AssetId,Data_LocalizedName_stringkey,Data_DifficultyType,Data_m_Faction,Data_m_Army,Data_m_Brain,Data_MaxHP,Data_OldWarhammerAgility,Data_OldWarhammerBallisticSkill,Data_OldWarhammerFellowship,Data_OldWarhammerIntelligence,Data_OldWarhammerPerception,Data_OldWarhammerStrength,Data_OldWarhammerToughness,Data_OldWarhammerWeaponSkill,Data_OldWarhammerWillpower,Data_WarhammerAgilitySetting_Modifier,Data_WarhammerBallisticSkillSetting_Modifier,Data_WarhammerFellowshipSetting_Modifier,Data_WarhammerIntelligenceSetting_Modifier,Data_WarhammerPerceptionSetting_Modifier,Data_WarhammerStrengthSetting_Modifier,Data_WarhammerToughnessSetting_Modifier,Data_WarhammerWeaponSkillSetting_Modifier,Data_WarhammerWillpowerSetting_Modifier,Data_WarhammerInitialAPBlue,Data_Body_m_PrimaryHand,Filepath,Unit_Name,PrimaryHand_Weapon
1508,5a4033955d0343979fb2a422aa3c065c,1b99545b-8423-42dd-8e28-a9b53cb4615d,Swarm,d64258e86eeb1d8479f35a9b16f6590a,02fbd2a876a24e0b8020deef7e009563,eeaf636459b8468083dc943e839dbf23,65,40,30,30,30,30,40,40,45,35,45,35,35,45,15,20,25,35,45,8,052a48be6dcf47d18abd96ed8ac77128,E:\Steam\RogueTrader\WorkFolder\Blueprints\Units\NPC\Unique\Human\Ch05_Epitaph\Ch05Inquisitor_MeleeSwarm_unit.jbp,Inquisition Techsorcist,Plasma Pistol


In [378]:
#search in units_data for Aurora
t = units_data.query('AssetId=="5a4033955d0343979fb2a422aa3c065c"')
#show only columns without nans
t[t.columns[~t.isnull().all()]]

,AssetId,Data_$type,Data_Alignment,Data_Body_DisableHands,Data_Body_m_PrimaryHand,Data_DifficultyType,Data_Gender,Data_IsCarefulShooter,Data_IsCheater,Data_IsStayOnSameSpotAfterCombat,Data_LocalizedName_stringkey,Data_MaxHP,Data_OldWarhammerAgility,Data_OldWarhammerBallisticSkill,Data_OldWarhammerFellowship,Data_OldWarhammerIntelligence,Data_OldWarhammerPerception,Data_OldWarhammerStrength,Data_OldWarhammerToughness,Data_OldWarhammerWeaponSkill,Data_OldWarhammerWillpower,Data_Size,Data_Skills_Awareness,Data_Skills_Coercion,Data_Skills_LoreWarp,Data_Skills_Medicae,Data_Skills_TechUse,Data_Speed_m_Value,Data_Subtype,Data_WarhammerAgilitySetting_Modifier,Data_WarhammerAgilitySetting_NotModified,Data_WarhammerAgilitySetting_isProfesional,Data_WarhammerBallisticSkillSetting_Modifier,Data_WarhammerBallisticSkillSetting_NotModified,Data_WarhammerBallisticSkillSetting_isProfesional,Data_WarhammerFellowshipSetting_Modifier,Data_WarhammerFellowshipSetting_NotModified,Data_WarhammerFellowshipSetting_isProfesional,Data_WarhammerInitialAPBlue,Data_WarhammerInitialAPYellow,Data_WarhammerIntelligenceSetting_Modifier,Data_WarhammerIntelligenceSetting_NotModified,Data_WarhammerIntelligenceSetting_isProfesional,Data_WarhammerMovementApPerCell,Data_WarhammerMovementApPerCellThreateningArea,Data_WarhammerPerceptionSetting_Modifier,Data_WarhammerPerceptionSetting_NotModified,Data_WarhammerPerceptionSetting_isProfesional,Data_WarhammerStrengthSetting_Modifier,Data_WarhammerStrengthSetting_NotModified,Data_WarhammerStrengthSetting_isProfesional,Data_WarhammerToughnessSetting_Modifier,Data_WarhammerToughnessSetting_NotModified,Data_WarhammerToughnessSetting_isProfesional,Data_WarhammerWeaponSkillSetting_Modifier,Data_WarhammerWeaponSkillSetting_NotModified,Data_WarhammerWeaponSkillSetting_isProfesional,Data_WarhammerWillpowerSetting_Modifier,Data_WarhammerWillpowerSetting_NotModified,Data_WarhammerWillpowerSetting_isProfesional,Data_m_AddFacts_0,Data_m_AddFacts_1,Data_m_AllowNonContextActions,Data_m_Army,Data_m_Brain,Data_m_Faction,Filepath
1508,5a4033955d0343979fb2a422aa3c065c,"fa4fa7e4548127a47a2846c91b051065, BlueprintUnit",TrueNeutral,False,052a48be6dcf47d18abd96ed8ac77128,Swarm,Male,False,False,False,1b99545b-8423-42dd-8e28-a9b53cb4615d,65,40,30,30,30,30,40,40,45,35,Medium,0,0,0,0,0,25.0,Default,45,False,False,35,False,False,35,False,False,8,6,45,False,False,1.0,3.0,15,False,False,20,False,False,25,False,False,35,False,False,45,False,False,de49d9600eeb4e9fb8a0adab3c46a260,33d4bf72f7b740779ed1ade0ac61e86b,False,02fbd2a876a24e0b8020deef7e009563,eeaf636459b8468083dc943e839dbf23,d64258e86eeb1d8479f35a9b16f6590a,E:\Steam\RogueTrader\WorkFolder\Blueprints\Units\NPC\Unique\Human\Ch05_Epitaph\Ch05Inquisitor_MeleeSwarm_unit.jbp


## Appendix

### Service cells

English version:

In [379]:
# search text by strinkey 7dd797bddb414f7684c7608bd7f1943f
index_df.query('index == "adc8d005-6cf8-4a9a-bacf-77fda627cff3"')['text']

35014    Grants +<u>wearer's {g|Encyclopedia:WarhammerIntelligence}INT{/g} {g|Encyclopedia:Bonus}bonus{/g}</u>% {g|Encyclopedia:DodgeGlossary}dodge{/g} to the wearer and their allies.
Name: text, dtype: object

In [385]:
#search filepath by file's asset id 
assets_df.query('AssetId=="33d4bf72f7b740779ed1ade0ac61e86b"')['File']

5058    e:\Steam\RogueTrader\WorkFolder\Blueprints\Equipment\CommonFeatures\Medicae15_Feature.jbp
Name: File, dtype: object

In [381]:
#search in units_data
units_data.query('AssetId=="491a465545bd41dca99c6319507827f8"')

,AssetId,Data_$type,Data_Alignment,Data_ArmyType,Data_Body_DisableHands,Data_Body_m_AdditionalLimbs_0,Data_Body_m_AdditionalLimbs_1,Data_Body_m_Armor,Data_Body_m_Belt,Data_Body_m_EmptyHandWeapon,Data_Body_m_Feet,Data_Body_m_Gloves,Data_Body_m_Head,Data_Body_m_Mechadendrites_0,Data_Body_m_Mechadendrites_1,Data_Body_m_Mechadendrites_2,Data_Body_m_Mechadendrites_3,Data_Body_m_Mechadendrites_4,Data_Body_m_Mechadendrites_5,Data_Body_m_Neck,Data_Body_m_PrimaryHand,Data_Body_m_PrimaryHandAlternative1,Data_Body_m_QuickSlots_0,Data_Body_m_QuickSlots_1,Data_Body_m_QuickSlots_2,Data_Body_m_QuickSlots_3,Data_Body_m_QuickSlots_4,Data_Body_m_Ring1,Data_Body_m_Ring2,Data_Body_m_SecondaryHand,Data_Body_m_SecondaryHandAlternative1,Data_Body_m_Shoulders,Data_Body_m_Wrist,Data_Components_0_$type,Data_Components_0_AbsorptionPenetration_Property,Data_Components_0_AbsorptionPenetration_PropertyName,Data_Components_0_AbsorptionPenetration_ValueRank,Data_Components_0_AbsorptionPenetration_ValueShared,Data_Components_0_AbsorptionPenetration_ValueType,Data_Components_0_Actions_Actions_0_$type,Data_Components_0_Actions_Actions_0_Actions_Actions_0_$type,Data_Components_0_Actions_Actions_0_Actions_Actions_0_BonusDamage_Property,Data_Components_0_Actions_Actions_0_Actions_Actions_0_BonusDamage_PropertyName,Data_Components_0_Actions_Actions_0_Actions_Actions_0_BonusDamage_ValueRank,Data_Components_0_Actions_Actions_0_Actions_Actions_0_BonusDamage_ValueShared,Data_Components_0_Actions_Actions_0_Actions_Actions_0_BonusDamage_ValueType,Data_Components_0_Actions_Actions_0_Actions_Actions_0_DamageType_Type,Data_Components_0_Actions_Actions_0_Actions_Actions_0_MaxDamage_Property,Data_Components_0_Actions_Actions_0_Actions_Actions_0_MaxDamage_PropertyName,Data_Components_0_Actions_Actions_0_Actions_Actions_0_MaxDamage_ValueRank,Data_Components_0_Actions_Actions_0_Actions_Actions_0_MaxDamage_ValueShared,Data_Components_0_Actions_Actions_0_Actions_Actions_0_MaxDamage_ValueType,Data_Components_0_Actions_Actions_0_Actions_Actions_0_MinDamage_Property,Data_Components_0_Actions_Actions_0_Actions_Actions_0_MinDamage_PropertyName,Data_Components_0_Actions_Actions_0_Actions_Actions_0_MinDamage_ValueRank,Data_Components_0_Actions_Actions_0_Actions_Actions_0_MinDamage_ValueShared,Data_Components_0_Actions_Actions_0_Actions_Actions_0_MinDamage_ValueType,Data_Components_0_Actions_Actions_0_Actions_Actions_0_Penetration_BonusValue_Property,Data_Components_0_Actions_Actions_0_Actions_Actions_0_Penetration_BonusValue_PropertyName,Data_Components_0_Actions_Actions_0_Actions_Actions_0_Penetration_BonusValue_ValueRank,Data_Components_0_Actions_Actions_0_Actions_Actions_0_Penetration_BonusValue_ValueShared,Data_Components_0_Actions_Actions_0_Actions_Actions_0_Penetration_BonusValue_ValueType,Data_Components_0_Actions_Actions_0_Actions_Actions_0_Penetration_DiceCountValue_Property,Data_Components_0_Actions_Actions_0_Actions_Actions_0_Penetration_DiceCountValue_PropertyName,Data_Components_0_Actions_Actions_0_Actions_Actions_0_Penetration_DiceCountValue_ValueRank,Data_Components_0_Actions_Actions_0_Actions_Actions_0_Penetration_DiceCountValue_ValueShared,Data_Components_0_Actions_Actions_0_Actions_Actions_0_Penetration_DiceCountValue_ValueType,Data_Components_0_Actions_Actions_0_Actions_Actions_0_Penetration_DiceType,Data_Components_0_Actions_Actions_0_Actions_Actions_0_PreRolledSharedValue,Data_Components_0_Actions_Actions_0_Actions_Actions_0_ResultSharedValue,Data_Components_0_Actions_Actions_0_Actions_Actions_0_UseDiceForDamage,Data_Components_0_Actions_Actions_0_Actions_Actions_0_Value_BonusValue_Property,Data_Components_0_Actions_Actions_0_Actions_Actions_0_Value_BonusValue_PropertyName,Data_Components_0_Actions_Actions_0_Actions_Actions_0_Value_BonusValue_Value,Data_Components_0_Actions_Actions_0_Actions_Actions_0_Value_BonusValue_ValueRank,Data_Components_0_Actions_Actions_0_Actions_Actions_0_Value_BonusValue_ValueShared,Data_Components_0_Actions_Actions_0_Actions_Actions_0_Valu

Finding a particular value in dataframe:

Russian version:

In [382]:
index_df_ru.query('index == "b3ed026f-d2d0-40cb-a54d-85d1c7f63507"')['text']

36273    В Просторе Коронус нет более достойно{mf|го|й} опекун{mf|а|ши} для обители веры, чем глава династии фон Валанциус. В столице мира в е{mf|го|е} честь будет возведена великолепная базилика, украшенная грандиозной парной скульптурой Императора и Вольного Торговца. Тысячи прихожан и жрецов будут пом...
Name: text, dtype: object

In [383]:
weapons_table.query('Item_Name=="Ripper Autopistol"')

,AssetId,Item_Name,Item_Descr,Add_Descr,Family,Is_Heavy,Holding_Type,Origin,Rarity,Damage_Type,Dodge_Penetration,Rate_Of_Fire,Min_Base_Damage,Max_Base_Damage,Max_Ammo,Max_Distance,Range,Penetration,Recoil,Cargo_Volume_Percent,Damage_Stat_Bonus_Factor,Ability_1_Name,Ability_1_Descr,Ability1_AP_Cost,Ability1_Type,Ability_2_Name,Ability_2_Descr,Ability2_AP_Cost,Ability2_Type,Ability_3_Name,Ability_3_Descr,Ability3_AP_Cost,Ability3_Type,Ability_4_Name,Ability_4_Descr,Ability4_AP_Cost,Ability4_Type,Ability_5_Name,Ability_5_Descr,Ability5_AP_Cost,Ability5_Type
184,151141ef419646ce9caa562670a898c3,Ripper Autopistol,NaN,Solid projectile weapons have an increased (+50%) overpenetration.,Solid,False,OneHanded,RangedWeaponry,Pattern,Piercing,0,8,3,5,16,10,Ranged,10,15,10,One,Pistol Shot,NaN,1,SingleShot,Pistol burst,Burst fire with equipped pistol.,2,Burst,Reload,Reloads equipped weapon.,1,Reload,NaN,NaN,0,NaN,NaN,NaN,0,NaN


________

### Lab

**Notes:**  
Toughness formula: (Base * (1 + 0.1* TGH bonus))

__________

### Saving

In [386]:
#units_df.to_csv(save_folder + r'\npc_units_stats.csv')


